# VibeVoice FastAPI Server on Google Colab 🚀

This notebook will clone `deepfak3voice` repository, install all necessary dependencies, and launch the FastAPI server using `uvicorn`. We'll then use `ngrok` to create a public URL so you can test the API endpoints from anywhere.

## Step 1: Setup & GPU Check

First, let's ensure you're using a GPU runtime. Go to **Runtime -> Change runtime type** and select **T4 GPU** as the hardware accelerator.

The command below will verify that the GPU is active.

In [ ]:
!nvidia-smi

## Step 2: Clone the Repository

Now, we'll clone your project from GitHub and move into the project directory.

In [ ]:
!git clone https://github.com/iamspruce/deepfak3voice.git
%cd deepfak3voice

## Step 3: Install Dependencies

We'll install all the packages listed in your `requirements.txt` file, plus `pyngrok` to manage the tunnel to our server.

In [ ]:
!pip install -q -r requirements.txt pyngrok

## Step 4: Configure Ngrok

To expose our server to the internet, we need an ngrok authtoken.

1.  Go to the [ngrok Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken) and sign up for a free account.
2.  Copy your authtoken.
3.  Paste the token into the form field below and run the cell.

In [ ]:
from pyngrok import ngrok
import os

NGROK_AUTH_TOKEN = "" #@param {type:"string"}

if not NGROK_AUTH_TOKEN:
    raise ValueError("Ngrok Authtoken is required. Please paste it into the form.")

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

## Step 5: Run the FastAPI Server! サーバーを起動する

This is the final step. We will start a public ngrok tunnel and then launch the uvicorn server. 

A public URL will be printed. **Click this URL** to access your API. The server logs will be displayed in this cell. To stop the server, just interrupt or stop this cell.

In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"🚀 Your VibeVoice server is live at: {public_url}")

# Run the FastAPI app
!uvicorn speak:app --host 0.0.0.0 --port 8000

## Step 6: How to Test Your API

Once the server is running, you can use `cURL` or any API client (like Postman) to test it. 

**⚠️ Reminder:** The audio generated will be a placeholder sine wave, not actual speech, until you implement the model inference logic in `speak.py`.

### Upload a Test Audio File
First, run the cell below to upload a sample `.wav` or `.mp3` file to use for voice cloning.

In [ ]:
from google.colab import files

uploaded = files.upload()

if uploaded:
    VOICE_FILE_PATH = list(uploaded.keys())[0]
    print(f"\nUploaded '{VOICE_FILE_PATH}' successfully.")
else:
    print("\nNo file uploaded.")

### Test with cURL
Now, create a new code cell and run the following commands. **Remember to replace `<YOUR_NGROK_URL>` with the public URL printed above.**

**1. Health Check**
```bash
!curl -X GET <YOUR_NGROK_URL>/health
```

**2. Single Speaker TTS**
```bash
# Make sure you have set the VOICE_FILE_PATH variable in the cell above
!curl -X POST -F "text=Hello world, this is a test." -F "voice_file=@{VOICE_FILE_PATH}" <YOUR_NGROK_URL>/single-speaker -o output.wav

# You can then play the audio in Colab:
from IPython.display import Audio
Audio('output.wav')
```